In [1]:
!git clone https://github.com/jjmachan/DeepHash
import DeepHash.datasets

Cloning into 'DeepHash'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 170 (delta 10), reused 18 (delta 6), pack-reused 143
Receiving objects: 100% (170/170), 12.98 MiB | 32.03 MiB/s, done.
Resolving deltas: 100% (52/52), done.


# Training Triplet Network using SemiHardNegetive

In [0]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision import models
import torch.nn as nn
from torch import optim
import numpy as np

from torch.optim import lr_scheduler
import torch.optim as optim
from torch.autograd import Variable

from DeepHash.trainer import fit
cuda = torch.cuda.is_available()

%matplotlib inline
import matplotlib
import matplotlib.pyplot as pl

from DeepHash.utils import freeze_model, list_trainable, del_last_layers, save, load, create_embeddings

In [3]:
batch_size = 64
num_workers = 1

#define transforms
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# load imagenet
image_dataset = {
        'train' :datasets.CIFAR10('./', train=True, download=True, transform=data_transforms['train']),
        'test' : datasets.CIFAR10('./', train=False, download=True, transform=data_transforms['train']) 
}

# Create the dataloaders
data_loader = {
    'train': torch.utils.data.DataLoader(image_dataset['train'], batch_size=batch_size, shuffle=True, num_workers=num_workers),
    'test': torch.utils.data.DataLoader(image_dataset['test'], batch_size=batch_size, shuffle=False, num_workers=num_workers)
}

  0%|          | 0/170498071 [00:00<?, ?it/s]

170500096it [00:01, 89374698.22it/s]                               


Extracting ./cifar-10-python.tar.gz to ./
Files already downloaded and verified


In [4]:
# An identity layer to pass the fc layer in resnet
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x
  
# Define model
resnet18  = models.resnet18(pretrained=True)
resnet18.fc = Identity()

# Freeze all the parameters in the model
def freeze_model(model):
  for params in model.parameters():
    params.requires_grad=False


Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 120MB/s] 


In [0]:
# Set up data loaders
from DeepHash.datasets import BalancedBatchSamplerCifar

cuda = torch.cuda.is_available()
train_batch_sampler = BalancedBatchSamplerCifar(image_dataset['train'].targets, n_classes=10, n_samples=25)
test_batch_sampler = BalancedBatchSamplerCifar(image_dataset['test'].targets, n_classes=10, n_samples=25)


In [0]:
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
online_train_loader = torch.utils.data.DataLoader(image_dataset['train'], batch_sampler=train_batch_sampler, **kwargs)
online_test_loader = torch.utils.data.DataLoader(image_dataset['test'], batch_sampler=test_batch_sampler, **kwargs)

In [0]:
loader = iter(online_train_loader)
images, labels = next(loader)
print(images.shape)

In [0]:
# Set up the network and training parameters
from DeepHash.networks import EmbeddingNet
from DeepHash.losses import OnlineTripletLoss
from DeepHash.utils import AllTripletSelector,HardestNegativeTripletSelector, RandomNegativeTripletSelector, SemihardNegativeTripletSelector # Strategies for selecting triplets within a minibatch
from DeepHash.metrics import AverageNonzeroTripletsMetric

margin = 1.
embedding_net = resnet18
model = embedding_net
if cuda:
    model.cuda()
loss_fn = OnlineTripletLoss(margin, SemihardNegativeTripletSelector(margin))
lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)
n_epochs = 20
log_interval = 15

In [12]:
fit(online_train_loader, online_test_loader, model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval, metrics=[AverageNonzeroTripletsMetric()])

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:100: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Train: [0/50000 (0%)]	Loss: 0.512598	Average nonzero triplets: 1471.0


KeyboardInterrupt: ignored

In [0]:
torch.save(model, './triplet_resnet18_SemihardNegetiveTripletSelector.mdl')

In [0]:
loss_fn = OnlineTripletLoss(margin, HardestNegativeTripletSelector(margin))
unfreeze_model(model)
fit(online_train_loader, online_test_loader, model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval, metrics=[AverageNonzeroTripletsMetric()])

In [0]:
torch.save(model, './triplet_vgg20_2.mdl')

In [0]:
# save to google drive 

from google.colab import drive
drive.mount('/gdrive')

!cp *.mdl /gdrive/My\ Drive/tooploox

In [0]:
# load data
from google.colab import drive
drive.mount('/gdrive')
!cp /gdrive/My\ Drive/tooploox/*.mdl ./ 

In [0]:
model_test = torch.load('./triplet_vgg19.mdl')
model.eval()

In [13]:
model_emb = model
freeze_model(model_emb)
print(model_emb)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [15]:
features, targets = create_embeddings(model_emb, data_loader, 512)

cuda
0
(64, 512)
error occured: 


In [0]:
# save the computed embeddings
save(features, targets, 'resnet18_semihardNegetive')

In [0]:
# save to google drive 

from google.colab import drive
drive.mount('/gdrive')

!cp *embs /gdrive/My\ Drive/tooploox

In [0]:
!ls /gdrive/My\ Drive/tooploox

# Classification loss

In [0]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

k_range = range(25,30)

def search_knn_accuracies(k_range, features, targets):
  acc = []
  for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(features['train'], targets['train'])
    #print('finished fitting')
    predict = knn.predict(features['test'][:300,:])
    #print('predicted')
    score = metrics.accuracy_score(targets['test'][:300], predict)
    print('K value: %d, accuracy: %0.7f' %(k, score))
    acc.append(score)
  print('Mean accuracy ',sum(acc)/len(acc))

# the best score was obtained when k = 20:24
search_knn_accuracies(k_range, features, targets)

# t-SNE

In [0]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
from sklearn.manifold import TSNE
%matplotlib inline

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import seaborn as sns
sns.set_style('darkgrid')
sns.set_palette('muted')
sns.set_context("notebook", font_scale=1.5,
                rc={"lines.linewidth": 2.5})
RS = 123
# need to create a subset of the data, too much time to process otherwise
x_subset = features['train'][:5000]
y_subset = targets['train'][:5000]

print(np.unique(y_subset))
labels = {
     0: 'airplane',  
     1: 'automobile',
     2: 'bird',
     3: 'cat',
     4: 'deer',
     5: 'dog',
     6: 'frog',
     7: 'horse',
     8: 'ship',
     9: 'truck',
}
# Utility function to visualize the outputs of PCA and t-SNE

def fashion_scatter(x, colors):
    # choose a color palette with seaborn.
    num_classes = len(np.unique(colors))
    palette = np.array(sns.color_palette("hls", num_classes))

    # create a scatter plot.
    f = plt.figure(figsize=(8, 8))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(x[:,0], x[:,1], lw=0, s=40, c=palette[colors.astype(np.int)])
    plt.xlim(-25, 25)
    plt.ylim(-25, 25)
    ax.axis('off')
    ax.axis('tight')

    # add the labels for each digit corresponding to the label
    txts = []

    for i in range(num_classes):

        # Position of each label at median of data points.

        xtext, ytext = np.median(x[colors == i, :], axis=0)
        txt = ax.text(xtext, ytext, str(labels[i]), fontsize=24)
        txt.set_path_effects([
            PathEffects.Stroke(linewidth=5, foreground="w"),
            PathEffects.Normal()])
        txts.append(txt)

    return f, ax, sc, txts

In [0]:
# do pca before passing to tsne to reduce noice and fast performance
time_start = time.time()

pca_50 = PCA(n_components=50)
pca_result_50 = pca_50.fit_transform(x_subset)

print('PCA with 50 components done! Time elapsed: {} seconds'.format(time.time()-time_start))

print('Cumulative variance explained by 50 principal components: {}'.format(np.sum(pca_50.explained_variance_ratio_)))
# perform tsne on 50 components
time_start = time.time()


fashion_pca_tsne = TSNE(random_state=RS).fit_transform(pca_result_50)

print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

In [0]:
fashion_scatter(fashion_pca_tsne, y_subset)